In [32]:
import pandas as pd
from binance.client import Client 
client = Client()

In [33]:
def getdata(symbol, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1m',start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

In [35]:
df = getdata('BTCUSDT', '2023-1-1')

In [36]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16538.45,16543.67,83.08143
2023-01-01 00:01:00,16543.04,16544.41,16538.48,16539.31,80.45300
2023-01-01 00:02:00,16539.31,16541.17,16534.52,16536.43,62.90197
2023-01-01 00:03:00,16536.43,16537.28,16531.00,16533.65,115.71894
2023-01-01 00:04:00,16534.12,16536.08,16527.51,16535.38,144.45369
...,...,...,...,...,...
2023-06-21 14:58:00,29469.99,29508.66,29468.86,29500.01,88.68509
2023-06-21 14:59:00,29500.00,29591.99,29496.00,29536.01,295.29346
2023-06-21 15:00:00,29536.02,29585.45,29532.00,29576.24,144.29076


In [37]:
df.resample('D').first().Open

Time
2023-01-01    16541.77
2023-01-02    16617.17
2023-01-03    16672.78
2023-01-04    16675.65
2023-01-05    16850.36
                ...   
2023-06-17    26345.01
2023-06-18    26516.99
2023-06-19    26339.98
2023-06-20    26844.35
2023-06-21    28308.00
Freq: D, Name: Open, Length: 172, dtype: float64

In [38]:
opens = df.resample('D').first().Open

In [39]:
def getlevels(date,first=True):
    if first:#if it is the first level return 
        return opens[date] * 0.998, opens[date] * 1.002
    else:
        return opens[date] * 0.996, opens[date] * 0.998

In [40]:
getlevels('2023-1-1')#buying and selling for first level

(16508.68646, 16574.85354)

In [41]:
# buying and selling for second level
getlevels('02023-1-1',first=False)

(16475.60292, 16508.68646)

In [42]:
def sliceDf(date):
    return df[df.index.date == pd.to_datetime(date)]

In [43]:
testDF = sliceDf(2023-1-1)

In [44]:
in_position = False
for index,row in testDF.iterrows():
    if not in_position:
        levels = getlevels('2023-1-1')
        if row.Low <= levels[0]:
            print(index)
            print('buy')
            in_position = True
    if in_position:
        if row.High >= levels[1]:
            print(index)
            print('sold')
            in_position = False

In [45]:
testDF['2023-1-1 00:35:00':]

,Open,High,Low,Close,Volume
Time,,,,,


In [46]:
testDF['2023-1-1 16:00:00':]#for selling

,Open,High,Low,Close,Volume
Time,,,,,


In [59]:
position_Arr = [False, False]
secondLevels = []
firstLevels = []
date = '2023-1-1'
testDF = sliceDf(date)

for index,row in testDF.iterrows():
    if not position_Arr[0]:
        firstLevels = getlevels(date)
        secondLevels = getlevels(date, first=False)
        if row.Low <= firstLevels[0]:
            print('buy')
            position_Arr[0] = True
    if position_Arr[0] and not position_Arr[1]:
        if row.Low <= secondLevels[0]:
            print('buy second')
            position_Arr[1] = True
        if row.High >= firstLevels[1]:
            print('sell')
            position_Arr[0] = False
    if position_Arr[1]:
        if row.High >= secondLevels[1]:
            print('sell second')
            position_Arr[1] = False
        

In [61]:
testDF.Low.min()

nan

In [62]:
testDF.dropna()

,Open,High,Low,Close,Volume
Time,,,,,


In [63]:
testDF.Low.min()

nan